In [72]:
from keras.layers import Input, LSTM, Dense, Conv1D, concatenate, Flatten, Dropout
from keras.models import Model
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

seed = 345045234

tf.random.set_seed(seed)
np.random.seed(seed)
plt.rcParams["figure.figsize"] = (20,10)

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

STEP = 10
past_history = 1000
future_target = 1

BATCH_SIZE = 6000

In [73]:
def multivariate_data(dataset_features, dataset_ground_truth, start_index, end_index, history_size,
                      target_size, step, single_step=False, print_index=False):
  data, labels, history = [], [], []

  start_index = start_index + history_size 
  # such that we always have history_size (100) observations to base our predictions on
  if end_index is None:
    end_index = len(dataset_features) - target_size 
    # such that we never predict based on the last future_target (10) measurements

  if print_index: print("start")
  for i in range(start_index, end_index): # start 100, end 790. 
      # TODO: It actually goes to 800 (when end_index is not None), meaning that our last 10 predictions cannot be verified with our training set. This is a problem
    if print_index: print("A", i,)
    indices = range(i-history_size, i, step) # range(0, 100) step size of 1          --- our sliding window
    data.append(dataset_features[indices]) # append new array that contains all values within our sliding window
    # TODO: ONE PROBLEM KINDA? Step size makes no sense. Doesn't feel like there's any point to it. 
    history.append(dataset_ground_truth[indices])

    if single_step:
      labels.append(dataset_ground_truth[i+target_size])
    else:
      labels.append(dataset_ground_truth[i:i+target_size])

  return np.array(data), np.array(labels), np.array(history)

In [74]:
#file_path = "..\\..\\Datasets\\BigData.xlsx"
#data_title = "data_test_raw_data"

file_path = "..\\..\\Datasets\\uniformdata.xlsx"
data_title = "data_uniformc_raw_data"

file_path3 = "..\\..\\Datasets\\uniformdata.xlsx"
data_title3 = "data_uniforma_raw_data"

file_path2 = "..\\..\\Datasets\\uniformdata.xlsx"
data_title2 = "data_uniformb_raw_data"

def thing(path,title):
    columns_data = ['1' ,'2', '3', '4', '5', '6', '7', '8', 'N/A_1', 'N/A_2', 'angle', 'time', 'session']
    columns_features_considered = columns_data[:8]
    column_ground_truth = columns_data[10]
    table_of_contents = pd.read_excel(path, sheet_name=0, header=None)
    sheet_index = table_of_contents[table_of_contents[0] == title][0].index[0]
    sheet_data = pd.read_excel(path, sheet_name=sheet_index + 1, header=None)
    sheet_data.columns = columns_data
    features = sheet_data[columns_features_considered]
    ground_truth = pd.DataFrame(sheet_data[column_ground_truth])
    features.plot(subplots=True)
    ground_truth.plot()
    dataset = features.values
    ground_truth2 = ground_truth.values
    xd, yd, _ = multivariate_data(dataset, ground_truth2, 0,
                                                   None, past_history,
                                                   future_target, STEP,
                                                   single_step=False, print_index = False)
    yd = np.squeeze(yd, axis=2)
    return xd, yd


In [75]:
x_train, y_train = thing(file_path, data_title)
x_train2, y_train2 = thing(file_path2, data_title2)
x_test, y_test = thing(file_path3,data_title3)

In [76]:
print ('Single window of past history : {}'.format(x_train[0].shape))
print(len(x_train), len(y_train))
print(x_train.shape)


In [77]:
#input
inp = int(past_history / STEP)

inputs = Input(shape=(inp,8), name='main_input')

conv = Conv1D(filters=50,kernel_size=int(inp/10),strides=2,padding='same',activation='relu')(inputs)

conv = Conv1D(filters=20,kernel_size=4,strides=2,padding='same',activation='relu')(conv)

conv = Conv1D(filters=10,kernel_size=4,strides=2,padding='same',activation='relu')(conv)

lstm = LSTM(32,return_sequences=False)(inputs) #true if deep lstm

#lstm = LSTM(32)(lstm)

x = Flatten()(conv)

#dense_1 = Dense(32,activation='relu',name='dense1')(x)

#dense_2 = Dense(32, activation='relu',name='dense2')(lstm)

auxiliary_output_1 = Dense(1, name='aux_output1')(x)

auxiliary_output_2 = Dense(1, name='aux_output2')(lstm)

print("success")

In [78]:
#intermediate layers
concat = concatenate([lstm,x])

x = Dense(64, activation='relu')(concat)
#x = Dropout(rate=0.5)(x)
#x = Dense(64, activation='relu')(x)
#x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

main_output = Dense(1,name='main_output')(x)

print('success')

In [79]:
#model

model = Model(inputs=[inputs], outputs=[main_output,auxiliary_output_1,auxiliary_output_2])

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01), loss={'main_output':'mae',
                                                                          'aux_output1':'mae',
                                                                          'aux_output2':'mae'},
              loss_weights={'main_output':1, 'aux_output1':0.3, 'aux_output2':0.3})


In [80]:
history = model.fit({'main_input': x_train},
          {'main_output': y_train, 'aux_output1': y_train, 'aux_output2': y_train},
          epochs=20, batch_size=BATCH_SIZE,validation_split=0.2)
print("succ")


In [81]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(loss))
  plt.figure()
  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()
  plt.show()

plot_train_history(history, 'Multi-Step Training and validation loss')

middle_res = model.predict(x_test,batch_size=BATCH_SIZE)

In [82]:
history = model.fit({'main_input': x_train2},
          {'main_output': y_train2, 'aux_output1': y_train2, 'aux_output2': y_train2},
          epochs=5, batch_size=BATCH_SIZE,validation_split=0.2)

plot_train_history(history, 'Multi-Step Training and validation loss')

print("succ")

In [83]:
print("start")
test_score = model.evaluate({'main_input': x_test},
          {'main_output': y_test, 'aux_output1': y_test, 'aux_output2': y_test},verbose=1,batch_size=1000)

print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

In [84]:
res = model.predict(x_test,batch_size=BATCH_SIZE)
res1 = res[0]
res2 = res[1]
res3 = res[2]
res4 = np.mean(res,axis=0)

sud = np.mean(middle_res, axis=0)

print("succ")

In [85]:
plt.plot(res1, label="1")
plt.plot(res2, label="2")
plt.plot(res3, label="3")
#plt.plot(res4, label="double")
#plt.plot(sud,label="single")
plt.plot(y_test, label="label")
plt.legend()
plt.show()